In [1]:
import os
os.chdir('../../' )
cwd=os.getcwd()

from oligo_designer_toolsuite.oligo_specificity_filter._filter_base import ProbeFilterBase
from oligo_designer_toolsuite.oligo_specificity_filter._filter_exact import ProbeFilterExact
from oligo_designer_toolsuite.oligo_specificity_filter._filter_blastn import ProbeFilterBlastn
from oligo_designer_toolsuite.oligo_specificity_filter._filter_bowtie import ProbeFilterBowtie

In [2]:
cwd

'/home/chels/CRISPR/repos/oligo-designer-toolsuite'

In [2]:
n_jobs=10
ligation_region=0
dir_output=cwd + "/output/"
dir_annotations=dir_output+"annotations/"
file_probe_info=dir_annotations + "oligo_DB_unknown_unknown_Custom_release_unknown_gene_transcript"
genes= ['WASH7P', 'DDX11L1', 'TRNT', 'NOC2L', 'PLEKHN1', 'AGRN','UBE2J2', 'DVL1', 'MIB2', 'LOC112268402_1' ]
min_probes_per_gene=2
file_transcriptome_fasta=dir_output +"annotations/reference_DB_unknown_unknown_Custom_release_unknown_genome_False_gene_transcript_True"
word_size=10
percent_identity=80
probe_length_min=30
probe_length_max=40
coverage=50


Test exact match filter

In [3]:
filter_exact=ProbeFilterExact(
        n_jobs,
        dir_output,
        file_probe_info,
        genes,
        dir_annotations)

In [4]:
filter_exact.apply()

INFO:filter_probes:Creating batches
INFO:filter_probes:Exact matches filtered in 2.4288301000015053 seconds


Test Blast filter

In [3]:
blast_filter=ProbeFilterBlastn(n_jobs,
        file_transcriptome_fasta, 
        dir_output,
        file_probe_info,
        genes, word_size,
        percent_identity, coverage, probe_length_min, probe_length_max, ligation_region)

TypeError: expected str, bytes or os.PathLike object, not int

In [7]:
blast_filter.apply()

Test Bowtie filter

In [5]:
n_jobs=10
min_mismatches=3

bowtie_filter=ProbeFilterBowtie(n_jobs,
        dir_output,
        file_probe_info,
        genes,
        probe_length_min,
        probe_length_max,
        file_transcriptome_fasta, min_mismatches)



In [6]:
bowtie_filter.apply()



# reads processed: 24
# reads with at least one reported alignment: 24 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 24 alignments to 1 output stream(s)
# reads processed: 426
# reads with at least one reported alignment: 426 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 468 alignments to 1 output stream(s)
# reads processed: 294
# reads with at least one reported alignment: 294 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 338 alignments to 1 output stream(s)
# reads processed: 671
# reads with at least one reported alignment: 671 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 6876 alignments to 1 output stream(s)
# reads processed: 339
# reads with at least one reported alignment: 339 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 574 alignments to 1 output stream(s)
# reads processed: 307
# reads with at least one reported alignment: 307 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 3017 alignments to 1 outp

INFO:filter_probes:Bowtie alignmnet search finished in 131.11580070000127 seconds


# reads processed: 1474
# reads with at least one reported alignment: 1474 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 8677 alignments to 1 output stream(s)


Empty DataFrame
Columns: [query, strand, reference, ref_start, query_sequence, read_quality, num_instances, mismatch_positions, query_gene_id, reference_gene_id, mismatch_positions_list]
Index: []
Empty DataFrame
Columns: [query, strand, reference, ref_start, query_sequence, read_quality, num_instances, mismatch_positions, query_gene_id, reference_gene_id, mismatch_positions_list]
Index: []


TypeError: 'float' object is not iterable